In [4]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd

class AutoDetectMultiOutputRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, estimators, param_spaces):
        self.estimators = estimators
        self.param_spaces = param_spaces

    def fit(self, X, y):
        self.models_ = []
        for i in range(y.shape[1]):
            best_score, best_estimator = -np.inf, None
            for estimator, params in zip(self.estimators, self.param_spaces):
                gs = GridSearchCV(estimator, params, cv=3, scoring='neg_mean_squared_error')
                gs.fit(X, y[:, i])
                if gs.best_score_ > best_score:
                    best_score, best_estimator = gs.best_score_, gs.best_estimator_
            self.models_.append(clone(best_estimator).fit(X, y[:, i]))
        return self

    def predict(self, X):
        return np.column_stack([model.predict(X) for model in self.models_])

# Generate synthetic multi-output data
X, y = make_regression(n_samples=300, n_features=8, n_targets=2, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define candidate models and parameter grids
estimators = [
    GaussianProcessRegressor(),
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42)
]

param_spaces = [
    {'alpha': [1e-10, 1e-2], 'kernel': [RBF(), Matern()]},
    {'n_estimators': [50, 100], 'max_depth': [3, 5, None]},
    {'n_estimators': [50, 100], 'max_depth': [3, 5]}
]

# Auto-detect best estimator per output and fit
model = AutoDetectMultiOutputRegressor(estimators, param_spaces)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse_scores = [mean_squared_error(y_test[:, i], y_pred[:, i]) for i in range(y.shape[1])]

results_df = pd.DataFrame({
    'Output': [f'Output {i}' for i in range(y.shape[1])],
    'MSE': mse_scores
})


In [5]:
results_df

,Output,MSE
0,Output 0,497.949955
1,Output 1,1795.017079


In [6]:
model

AutoDetectMultiOutputRegressor(estimators=[GaussianProcessRegressor(),
                                           RandomForestRegressor(random_state=42),
                                           GradientBoostingRegressor(random_state=42)],
                               param_spaces=[{'alpha': [1e-10, 0.01],
                                              'kernel': [RBF(length_scale=1),
                                                         Matern(length_scale=1, nu=1.5)]},
                                             {'max_depth': [3, 5, None],
                                              'n_estimators': [50, 100]},
                                             {'max_depth': [3, 5],
                                              'n_estimators': [50, 100]}])

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from hyperopt import hp
import ray


# ✅ Wrapper class
class MultiTargetRegressorWithStd(BaseEstimator, RegressorMixin):
    def __init__(self, estimator_configs):
        self.estimator_configs = estimator_configs
        self.models_ = []

    def fit(self, X, Y):
        self.models_ = []
        for i, config in enumerate(self.estimator_configs):
            model = self._make_estimator(config)
            model.fit(X, Y[:, i])
            self.models_.append(model)
        return self

    def predict(self, X, return_std=False):
        preds = []
        stds = []

        for model in self.models_:
            if return_std:
                try:
                    pred, std = model.predict(X, return_std=True)
                except TypeError:
                    pred = model.predict(X)
                    std = self._estimate_std(model, X)
                preds.append(pred)
                stds.append(std)
            else:
                preds.append(model.predict(X))

        Y_pred = np.stack(preds, axis=1)
        if return_std:
            Y_std = np.stack(stds, axis=1)
            return Y_pred, Y_std
        return Y_pred

    def _estimate_std(self, model, X):
        """For RandomForest: use std dev of tree predictions"""
        if hasattr(model, "estimators_"):
            all_preds = np.array([tree.predict(X) for tree in model.estimators_])
            return np.std(all_preds, axis=0)
        else:
            return np.full(X.shape[0], np.nan)

    def _make_estimator(self, config):
        t = config["type"]
        if t == "gp":
            return GaussianProcessRegressor(
                kernel=RBF(length_scale=config["length_scale"]),
                alpha=config["alpha"],
                normalize_y=True
            )
        elif t == "bayesian_ridge":
            return BayesianRidge(alpha_1=config["alpha_1"], alpha_2=config["alpha_2"])
        elif t == "rf":
            return RandomForestRegressor(
                n_estimators=config["n_estimators"],
                max_depth=config["max_depth"]
            )
        else:
            raise ValueError(f"Unsupported estimator type: {t}")

    def get_params(self, deep=True):
        return {"estimator_configs": self.estimator_configs}

    def set_params(self, **params):
        if "estimator_configs" in params:
            self.estimator_configs = params["estimator_configs"]
        return self


# 🔧 Search space
def get_search_space(n_targets):
    space = {}
    for i in range(n_targets):
        space[f"model_{i}"] = hp.choice(f"model_{i}", [
            {
                "type": "gp",
                "length_scale": hp.uniform(f"gp_ls_{i}", 0.1, 10.0),
                "alpha": hp.loguniform(f"gp_alpha_{i}", -10, -1),
            },
            {
                "type": "bayesian_ridge",
                "alpha_1": hp.loguniform(f"br_a1_{i}", -10, -1),
                "alpha_2": hp.loguniform(f"br_a2_{i}", -10, -1)
            },
            {
                "type": "rf",
                "n_estimators": hp.choice(f"rf_n_{i}", [50, 100, 200]),
                "max_depth": hp.choice(f"rf_d_{i}", [3, 5, 7])
            }
        ])
    return space


def train_model(config):
    model_configs = [config[f"model_{i}"] for i in range(Y.shape[1])]
    model = MultiTargetRegressorWithStd(model_configs)
    model.fit(X_train, Y_train)
    Y_pred, Y_std = model.predict(X_val, return_std=True)
    mse = mean_squared_error(Y_val, Y_pred)
    tune.report({"mse": mse})


# 📊 Sample data
X, Y = make_regression(n_samples=300, n_features=5, n_targets=3, noise=0.5, random_state=0)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


# 🚀 Launch hyperopt + ray
def run_search():
    ray.init(ignore_reinit_error=True)

    space = get_search_space(Y.shape[1])
    algo = HyperOptSearch(space, metric="mse", mode="min")

    tuner = tune.Tuner(
        train_model,
        tune_config=tune.TuneConfig(
            search_alg=algo,
            num_samples=12,
        ),
    )
    results = tuner.fit()
    best = results.get_best_result(metric="mse", mode="min")
    print("Best Config:", best.config)
    print("Best MSE:", best.metrics["mse"])
    ray.shutdown()


if __name__ == "__main__":
    run_search()
